Prévision de la mortalité (France, 2000–2022)

Ce projet étudie la mortalité de la population française sur la période 2000–2022.

I - Construction des données:

 Les données exploitées dans cette étude proviennent de la Human Mortality Database
 (https://www.mortality.org), plateforme spécialisée dans la collecte et la diffusion de
 statistiques démographiques longitudinales. Après inscription, nous avons pu accéder à deux
 fichiers essentiels concernant la population française.